# Datenvorbereitung

Die Warnungen, die vom Sytsem automatisch bei z.B. veralteten Versionen erstellt werden, werden deaktiviert.

In [5]:
import warnings
warnings.filterwarnings('ignore')

Einladen der Pakte, die für die Vorbereitung der Daten benötigt werden. Hierzu wird Geopandas verwendet.

In [2]:
import geopandas as gpd

Einladen der Daten, die vobereitet werden. Das sind die Standorte der Ärzte für ganz Deutschland (arzt). Dieser stammt aus dem ESRI-OpenDataPortal. Außerdem der Anteil der Ü64-Jährigen pro Gemeinde in Deutschland (anteil) vom Statistikportal des Bundes und der Länder. Auch die Gemeindegrenzen mit der Einwohnerzahl werden eingeladen (gemeinde). Die Daten der Ärzte und der Gemeindegenzen mit Einwohnerzahl werden noch ins EPSG 25832 transformiert, da diese in 4326 vorliegen.

In [33]:
arzt = gpd.read_file('./Daten/Aerzte/aerzte_deutschland.geojson')
arzt = arzt.to_crs(epsg = 25832)
anteil = gpd.read_file('./Daten/Bevoelkerung/Anteil_Ue64_Deutschland.geojson')
gemeinde = gpd.read_file('./Daten/Bevoelkerung/Gemeindegrenzen_2021_mit_Einwohnerzahl.geojson')
gemeinde = gemeinde.to_crs(epsg = 25832)

Einladen der Bundesländer von Deutschland. Daraus wird Schleswig-Holstein herausgefiltert. Diese Geometrie dient als Ausgangspunkt, auf die die andern zwei Datensätze zugeschnitten werden. Die Koordinaten werden ins Koordinatensstem 25832 transformiert.

In [8]:
sh = gpd.read_file('./Daten/BundeslA4nder_2021_mit_Einwohnerzahl.geojson')
gdf = sh[sh['GEN'] == 'Schleswig-Holstein']
gdf = gdf.to_crs(epsg = 25832)

Die Daten (arzt und anteil) werden beide auf Schleswig-Holstein (gdf) zugeschnitten. 

In [34]:
arzt = gpd.clip(arzt, gdf)
anteil = gpd.clip(anteil, gdf)
gemeinde = gpd.clip(gemeinde, gdf)

Der Datensatz der Ärzte wird gefiltert. Es werden nur Allgemeinärzte (healthcare_speciality = general) und undefinierte Ärzte (healthcare_speciality = NaN) berücksichtigt.

In [20]:
# Auswahl der Zeilen mit "general" oder Null in der Spalte "healthcare_speciality"
auswahl = arzt.loc[(arzt['healthcare_speciality'].str.contains('general', case=False, na=False)) | (arzt['healthcare_speciality'].isna())]

Die zugeschnittenen Datensätze werden für weitere Berechnungen abgespeichert.

In [38]:
auswahl.to_file('./Daten/Aerzte/aerzte_general_sh.geojson')
anteil.to_file('./Daten/Bevoelkerung/Anteil_Ue64_SH.geojson')
gemeinde.to_file('./Daten/Bevoelkerung/Gemeindegrenzen_SH_2021_mit_Einwohnerzahl.geojson')